In [1]:
import ast
import datetime
import re

import boto3
from envs import env
from jose import JWTError, jwt
import requests
from getpass import getpass

from aws_srp import AWSSRP
from exceptions import TokenVerificationException

from config import config

import uuid

In [2]:
class Garner:
    
    #user_class = UserObj
    #group_class = GroupObj
    
    def __init__(self, username=None, private_key=None, password=None):
        
        self.user_pool_id = config["aws_user_pools_id"]
        self.user_pool_region = config["aws_cognito_region"]
        self.client_id = config["aws_user_pools_web_client_id"]
        
        self.username = username
        self.private_key = private_key
        
        self.refresh_token = None
        self.token_type = None
        self.access_token = None
        self.pool_jwk = None
        self.id_token = None
        
        self.client = boto3.client("cognito-idp", region_name=self.user_pool_region)
        
        self.authenticate(password)
        
    def authenticate(self, password):
        """
        Authenticate the user using the SRP protocol
        :param password: The user's passsword
        :return:
        """
        if not password:
            print("Enter Password")
        aws = AWSSRP(
            username=self.username,
            password=(password if password else getpass("Password:")),
            pool_id=self.user_pool_id,
            client_id=self.client_id,
            client=self.client,
        )
        tokens = aws.authenticate_user('Password:')
        print("Authenticated")
        self.verify_token(tokens["AuthenticationResult"]["IdToken"], "id_token", "id")
        self.refresh_token = tokens["AuthenticationResult"]["RefreshToken"]
        self.verify_token(
            tokens["AuthenticationResult"]["AccessToken"], "access_token", "access"
        )
        self.token_type = tokens["AuthenticationResult"]["TokenType"]
        self.access_token = tokens['AuthenticationResult']['AccessToken']
        
    def get_keys(self):
        if self.pool_jwk:
            return self.pool_jwk

        # Check for the dictionary in environment variables.
        pool_jwk_env = env("COGNITO_JWKS", {}, var_type="dict")
        if pool_jwk_env:
            self.pool_jwk = pool_jwk_env
        # If it is not there use the requests library to get it
        else:
            self.pool_jwk = requests.get(
                "https://cognito-idp.{}.amazonaws.com/{}/.well-known/jwks.json".format(
                    self.user_pool_region, self.user_pool_id
                )
            ).json()
        return self.pool_jwk

    def get_key(self, kid):
        keys = self.get_keys().get("keys")
        key = list(filter(lambda x: x.get("kid") == kid, keys))
        return key[0]
    
    def verify_token(self, token, id_name, token_use):
        kid = jwt.get_unverified_header(token).get("kid")
        unverified_claims = jwt.get_unverified_claims(token)
        token_use_verified = unverified_claims.get("token_use") == token_use
        if not token_use_verified:
            raise TokenVerificationException("Your {} token use could not be verified.")
        hmac_key = self.get_key(kid)
        try:
            verified = jwt.decode(
                token,
                hmac_key,
                algorithms=["RS256"],
                audience=unverified_claims.get("aud"),
                issuer=unverified_claims.get("iss"),
            )
        except JWTError:
            raise TokenVerificationException("Your {} token could not be verified.")
        setattr(self, id_name, token)
        return verified
    
    def check_token(self, renew=True):
        """
        Checks the exp attribute of the access_token and either refreshes
        the tokens by calling the renew_access_tokens method or does nothing
        :param renew: bool indicating whether to refresh on expiration
        :return: bool indicating whether access_token has expired
        """
        if not self.access_token:
            raise AttributeError('Access Token Required to Check Token')
        now = datetime.datetime.now()
        dec_access_token = jwt.get_unverified_claims(self.access_token)

        if now > datetime.datetime.fromtimestamp(dec_access_token['exp']):
            expired = True
            if renew:
                self.renew_access_token()
        else:
            expired = False
        return expired
    
    def renew_access_token(self):
        """
        Sets a new access token on the User using the refresh token.
        """
        auth_params = {'REFRESH_TOKEN': self.refresh_token}
        #self._add_secret_hash(auth_params, 'SECRET_HASH')
        refresh_response = self.client.initiate_auth(
            ClientId=self.client_id,
            AuthFlow='REFRESH_TOKEN',
            AuthParameters=auth_params,
        )

        status_code = refresh_response.get(
            'HTTPStatusCode',
            refresh_response['ResponseMetadata']['HTTPStatusCode']
        )
        
        if status_code == 200:
            self.access_token = refresh_response['AuthenticationResult']['AccessToken']
            self.id_token: refresh_response['AuthenticationResult']['IdToken']
            self.token_type: refresh_response['AuthenticationResult']['TokenType']

In [3]:
u = Garner('e80cb5db-788a-4a84-9604-513cb3e8152b', 'dbfaab17-b8b1-41ad-95ec-1568ac734109')

Enter Password


Password: ·········


Authenticated


In [4]:
u.check_token()
u.private_key

'dbfaab17-b8b1-41ad-95ec-1568ac734109'

In [5]:
import requests
import json

APPSYNC_API_ENDPOINT_URL = config["aws_appsync_graphqlEndpoint"]
#APPSYNC_API_KEY = "da2-fakeApiId123456"

headers = {
    #'Content-Type': "application/graphql",
    #'x-api-key': APPSYNC_API_KEY,
    #'Authorization': u.pool_jwk
    'Authorization': str(u.access_token)
}

params = {
    "filter": {"privateKey" : {"eq": u.private_key}}
}

def execute_gql(query):
    u.check_token()
    payload_obj = {"query": query, "variables" : params,}
    payload = json.dumps(payload_obj)
    response = requests.request("POST", APPSYNC_API_ENDPOINT_URL, data=payload, headers=headers)
    return response

query = """query ListPools(
    $filter: ModelPoolFilterInput
    $limit: Int
    $nextToken: String
  ) {
    listPools(filter: $filter, limit: $limit, nextToken: $nextToken) {
      items {
        id
        title
        privateKey
      }
    }
  }"""

output = execute_gql(query).json()

#print(output)
print("working on: {}".format('\033[1m' +output['data']['listPools']['items'][0]['title']))
#for item in output['data']['listPools']['items']:
   # print("-{}".format(item['title'][]))

working on: kai man


In [6]:
object_name = 'movie.gif'
s3 = boto3.client('s3')

with open(object_name, 'rb') as data:
    loc = 'pooldata211140-dev'
    
    key = 'protected/'
    key += u.username
    key += '/'
    key += str(uuid.uuid4())
    key += '.'
    key += object_name.split('.')[-1]
    
    s3.upload_fileobj(data, loc, key)

In [7]:
import logging
import boto3
from botocore.exceptions import ClientError


def create_presigned_post(bucket_name, object_name,
                          fields=None, conditions=None, expiration=3600):
    """Generate a presigned URL S3 POST request to upload a file

    :param bucket_name: string
    :param object_name: string
    :param fields: Dictionary of prefilled form fields
    :param conditions: List of conditions to include in the policy
    :param expiration: Time in seconds for the presigned URL to remain valid
    :return: Dictionary with the following keys:
        url: URL to post to
        fields: Dictionary of form fields and values to submit with the POST
    :return: None if error.
    """

    # Generate a presigned S3 POST URL
    s3_client = boto3.client('s3')
    try:
        response = s3_client.generate_presigned_post(bucket_name,
                                                     object_name,
                                                     Fields=fields,
                                                     Conditions=conditions,
                                                     ExpiresIn=expiration)
    except ClientError as e:
        logging.error(e)
        return None

    # The response contains the presigned URL and required fields
    return response

#Add function to reach lambda
#https://github.com/SanderKnape/aws-upload-to-s3/blob/master/src/App.js

In [8]:
import requests    # To install: pip install requests

# Generate a presigned S3 POST URL
object_name = 'OBJECT_NAME'
response = create_presigned_post('BUCKET_NAME', object_name)
if response is None:
    exit(1)

# Demonstrate how another Python program can use the presigned URL to upload a file
with open(object_name, 'rb') as f:
    files = {'file': (object_name, f)}
    http_response = requests.post(response['url'], data=response['fields'], files=files)
# If successful, returns HTTP status code 204
logging.info(f'File upload HTTP status code: {http_response.status_code}')

AttributeError: 'NoneType' object has no attribute 'access_key'